In [84]:
# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, LayerNormalization, Dense, TimeDistributed, RepeatVector, MultiHeadAttention, Concatenate, Bidirectional, Attention, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from keras_tuner import HyperParameters, Hyperband
import tensorflow.keras.backend as K

# Carregar dados
data = pd.read_csv("dataset.csv")

# Converter 'id' em datetime e definir como índice
data['timestamp'] = pd.to_datetime(data['id'], errors='coerce')
data.set_index('timestamp', inplace=True)

# Selecionar variáveis relevantes
variables = data[['ws100', 'humid', 'wdisp100']]

# Padronização dos dados usando MinMaxScaler
scaler = MinMaxScaler()
variables_scaled = scaler.fit_transform(variables)

# Parâmetros para sequências de entrada e saída
sequence_length = 36   # Janela de aprendizado de 36 passos de tempo
step_ahead = 6         # Previsão para o sexto passo (1 hora de antecedência)
split_ratio = 0.80      # 80% para treinamento e 20% para teste

# Divisão dos dados em treinamento e teste
split_index = int(len(variables_scaled) * split_ratio)
train_data = variables_scaled[:split_index]
test_data = variables_scaled[split_index:]

# Função para preparar sequências de dados para previsão de um passo específico
def create_sequences_single_ahead(data, seq_length, step_ahead=6):
    X, y = [], []
    for i in range(len(data) - seq_length - step_ahead + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i + seq_length + step_ahead -1, 0])  # ws100 é a primeira coluna
    return np.array(X), np.array(y)

# Criar sequências para treinamento e teste
X_train, y_train = create_sequences_single_ahead(train_data, sequence_length, step_ahead)
X_test, y_test = create_sequences_single_ahead(test_data, sequence_length, step_ahead)

# Reshape de y_train e y_test para serem compatíveis com a saída do modelo
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

# Valores mínimos e máximos de 'ws100' para inversão da escala
ws100_min = scaler.data_min_[0]
ws100_max = scaler.data_max_[0]

# Número de características
num_features = X_train.shape[2]

# Verificação das formas dos dados
print(f"X_train shape: {X_train.shape}")  # Esperado: (num_samples, 36, 5)
print(f"y_train shape: {y_train.shape}")  # Esperado: (num_samples, 1)
print(f"X_test shape: {X_test.shape}")    # Esperado: (num_samples, 36, 5)
print(f"y_test shape: {y_test.shape}")    # Esperado: (num_samples, 1)")

def augment_data(X, y, num_augmented_samples=50):
    X_augmented = []
    y_augmented = []
    for i in range(len(y)):
        # Inversão da normalização de y
        y_inv = y[i] * (ws100_max - ws100_min) + ws100_min
        if y_inv < 6.0:
            for _ in range(5):  # Aumentar 5 vezes cada amostra
                noise = np.random.normal(0, 0.01, X[i].shape)
                X_augmented.append(X[i] + noise)
                y_augmented.append(y[i])
    return np.array(X_augmented), np.array(y_augmented)

# Aplicar data augmentation nos dados de treinamento
X_train_aug, y_train_aug = augment_data(X_train, y_train)
X_train = np.concatenate((X_train, X_train_aug), axis=0)
y_train = np.concatenate((y_train, y_train_aug), axis=0)

X_train shape: (6007, 36, 3)
y_train shape: (6007, 1)
X_test shape: (1472, 36, 3)
y_test shape: (1472, 1)


Definição de funções de perdas customizadas.

Geração do modelo usando LSTM bidirecional

In [85]:
def build_model_cnn(hp):
    # Hiperparâmetros com espaços de busca reduzidos
    filters = hp.Int('filters', min_value=16, max_value=256, step=16)
    kernel_size = hp.Choice('kernel_size', values=[3, 5, 7])
    pool_size = hp.Choice('pool_size', values=[2, 3])
    units = hp.Int('units', min_value=16, max_value=256, step=16)
    learning_rate = hp.Choice('learning_rate', [1e-4, 1e-3, 1e-2])
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop'])
    num_heads = hp.Int('num_heads', min_value=2, max_value=6, step=2)
    
    # Seleção do Otimizador
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    # Definir ws100_min e ws100_max como constantes do Keras
    ws100_min_k = K.constant(ws100_min, dtype='float32')
    ws100_max_k = K.constant(ws100_max, dtype='float32')

    # Definir a função de perda personalizada dentro da função
    def custom_loss(y_true, y_pred):
        # Inversão da normalização para obter valores originais
        y_true_inv = y_true * (ws100_max_k - ws100_min_k) + ws100_min_k
        y_pred_inv = y_pred * (ws100_max_k - ws100_min_k) + ws100_min_k

        # Calcula o erro absoluto
        error = K.abs(y_true_inv - y_pred_inv)

        # Cria um tensor booleano indicando onde y_true_inv < 6 m/s
        condition = K.less(y_true_inv, 6.0)

        # Define um peso maior para velocidades abaixo de 6 m/s
        greater_weight = 5.0  # Peso maior para erros quando y_true_inv < 6 m/s
        lesser_weight = 1.0   # Peso normal para outros casos

        # Aplica os pesos
        weights = K.switch(condition, greater_weight, lesser_weight)
        weighted_error = weights * error

        # Retorna a média do erro ponderado
        return K.mean(weighted_error)
    
    encoder_inputs = tf.keras.Input(shape=(sequence_length, num_features), name='encoder_input')
    x = encoder_inputs

    # Camada CNN
    x = Conv1D(
        filters=filters,
        kernel_size=kernel_size,
        activation='relu',
        padding='same',
        kernel_regularizer=l2(1e-4),
        name='conv1d_layer'
    )(x)

    # Camada de Pooling
    x = MaxPooling1D(pool_size=pool_size, padding='same', name='maxpool_layer')(x)

    # Camada LSTM Bidirecional
    x = Bidirectional(LSTM(
        units=units,
        return_sequences=True,
        activation='tanh',
        dropout=dropout_rate,
        kernel_regularizer=l2(1e-3),
        name='bidirectional_lstm_layer'
    ))(x)
    x = LayerNormalization(name='lstm_norm')(x)

    # Encoder State
    encoder_state = x[:, -1, :]

    # Decoder Input
    decoder_inputs = RepeatVector(1, name='repeat_vector')(encoder_state)
    decoder_outputs = decoder_inputs

    # Camada LSTM no Decoder
    decoder_outputs = LSTM(
        units=units,
        return_sequences=True,
        activation='tanh',
        dropout=dropout_rate,
        kernel_regularizer=l2(1e-3),
        name='decoder_lstm_layer'
    )(decoder_outputs)
    decoder_outputs = LayerNormalization(name='decoder_lstm_norm')(decoder_outputs)

    # Atenção Customizada
    # Criar máscara de atenção baseada em velocidades abaixo de 6 m/s
    ws100_sequence = x[:, :, 0:1]  # Seleciona a primeira característica e mantém a dimensão
    ws100_sequence_inv = ws100_sequence * (ws100_max_k - ws100_min_k) + ws100_min_k

    attention_weights = tf.where(ws100_sequence_inv < 6.0, 5.0, 1.0)
    # Normalizar os pesos de atenção
    attention_weights = attention_weights / K.sum(attention_weights, axis=1, keepdims=True)

    # Aplicar a máscara de atenção
    x_weighted = x * attention_weights

    # Camada de Atenção
    attention_output = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=units,
        name='multi_head_attention'
    )(
        query=decoder_outputs,
        key=x_weighted,
        value=x_weighted
    )

    attention_output = Dropout(dropout_rate, name='attention_dropout')(attention_output)
    decoder_concat_input = Concatenate(axis=-1, name='concat_attention')([decoder_outputs, attention_output])

    # Camada de Saída
    outputs = TimeDistributed(Dense(1, activation='linear'), name='output_layer')(decoder_concat_input)

    # Definição do Modelo
    model = Model(inputs=encoder_inputs, outputs=outputs, name='Wind_Speed_Predictor_CNN_BiLSTM_Attention')

    # Compilação do Modelo com Função de Loss Personalizada
    model.compile(
        optimizer=optimizer,
        loss=custom_loss,
        metrics=[
            tf.keras.metrics.RootMeanSquaredError(name='rmse'),
            tf.keras.metrics.MeanAbsoluteError(name='mae')
        ]
    )

    return model

In [86]:
# Definir o tuner usando Hyperband
tuner = Hyperband(
    build_model_cnn,
    objective='val_loss',
    max_epochs=20,
    factor=3,
    directory='tuner_dir',
    project_name='bidirectional_lstm_tuning_updated'
)

# Callback para parada antecipada
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Iniciar a busca de hiperparâmetros
tuner.search(
    X_train,
    y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Obter os melhores hiperparâmetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Construir o melhor modelo com os hiperparâmetros encontrados
model = tuner.hypermodel.build(best_hps)

# Treinar o melhor modelo
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Salvar o modelo final
model_save_path_final = 'best_bidirectional_lstm_model.h5'
model.save(model_save_path_final)
print(f"Modelo salvo em: {model_save_path_final}")

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


Hypertunning do modelo para gerar os melhores parêmetros

In [ ]:
# %% [markdown]
# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Inversão da padronização
y_pred_inv = y_pred * (ws100_max - ws100_min) + ws100_min
y_test_inv = y_test * (ws100_max - ws100_min) + ws100_min

# Converter para classes: 1 se y < 6 m/s, 0 caso contrário
y_pred_class = (y_pred_inv < 6.0).astype(int)
y_test_class = (y_test_inv < 6.0).astype(int)

print(classification_report(y_test_class, y_pred_class))

# Plotagem das previsões vs valores reais focando em y < 6 m/s
indices_below_6 = np.where(y_test_inv < 6.0)[0]

plt.figure(figsize=(12,6))
plt.plot(indices_below_6, y_test_inv[indices_below_6], label='Valor Real')
plt.plot(indices_below_6, y_pred_inv[indices_below_6], label='Previsão')
plt.legend()
plt.title('Comparação entre Valores Reais e Previstos para y < 6 m/s')
plt.xlabel('Amostras')
plt.ylabel('Velocidade do Vento a 100 metros')
plt.show()

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


ValueError: Found array with dim 3. None expected <= 2.

Avaliação e plotagem dos gráficos.

## O que fazer:

Você tem que falar sobre LSTM, LSTM bidirecional, LSTM encoder-decoder, Loss customizáveis, método automático de seleção do loss.
